# Where Clause... to do
In the `02_Demystifying_table_sharding_tablets_and_data_distribution.ipynb` notebook, you discovered how YugabyteDB stores data in tablets. You also learned that YugabyteDB and YSQL support two sharding strategies for tables: hash and range sharding.

An index in YugabyteDB is also distributed. Just like a table, YugabyteDB stores and distributes the data for an index stores in one or more tablets.  An index can employ either a hash or range sharding strategy. 

> Note
> 
> In this regard, a tablet represents a shard of data which contains a set of rows for a logical entity. Each tablet is a customized RocksDB instance. A tablet leader has a peer group known as tablet followers, and this group of tablet peers exists as a Raft consensus group. YugabyteDB calls this distributed document store, DocDB.

In this notebook, using Explain Plans, built-in functions, and custom utilities for YB-TServer metrics, you will learn how YugabyteDB stores data for an index in one or more tablets. You will also learn how YugabyteDB reads tablet data during query execution.


## Requirements
Before running the cells in this notebook, you must first execute all the cells in the following notebook:
- `01_Lab_Setup.ipynb`

## Getting help
The best way to get help from the Yugabyte University team is to post your question on YugabyteDB Community Slack in the #training or #yb-university channels. To sign up, visit [https://communityinviter.com/apps/yugabyte-db/register](https://communityinviter.com/apps/yugabyte-db/register).

## Connect to the `db_ybu` database
Run all the cells in this section:
- Connect using Python and PostgreSQL driver
- Load the SQL magic extension
- Create the prepared statements
- View the listener
- View the DDL for tbl_cities

In [90]:
# connect use Python 3.7.9+
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
# Use %store -r to read 01_Lab_Requirements_Setup variables

%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_GITPOD_WORKSPACE_URL

db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME


connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

### Load the SQL magic extension

In [91]:
%reload_ext sql

# SQL magic for python connection string
%sql {connection_str}

### Create the prepared statements

In [92]:
#%% python, but prepared statements as sql magic
if (MY_GITPOD_WORKSPACE_URL is None):
    a = %sql select fn_yb_create_stmts()
else:
    WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
    a = %sql select fn_yb_create_stmts(:WORKSPACE_URL)

print (a)

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
+----------------------------------+
|        fn_yb_create_stmts        |
+----------------------------------+
| 2022-07-27 21:10:02.853390-07:00 |
+----------------------------------+


Confirm that the following query returns a count of 3 (for three prepared statements).

In [93]:
%%sql 
select count(*) from pg_prepared_statements where 1=1 and name in ('stmt_util_metrics_snap_tablet','stmt_util_metrics_snap_table','stmt_util_metrics_snap_reset')

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


count
3


### View the listener address
Run the following cell to view the host for your client connection

In [ ]:
%%sql /* confirm listener */
show listen_addresses;

### View DDL for tbl_cities

Run the following cell to execute the describe table command, `\d`.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # \d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ tbl_cities"
# ./bin/ysqlsh -d ${DB_NAME} -c "\d+ idx_cities_city_name_range"

---
## q1 | Create a secondary index using range sharding
When you create an index for a table with a sort order (`asc` or `desc`), YugabyteDB will create the index using a range sharding strategy.
  

In [ ]:
%%sql
drop index if exists idx_cities_city_name_range;
drop index if exists idx_cities_city_name_hash;

select pg_sleep(1);

create index idx_cities_city_name_range on tbl_cities (city_name asc);

Review the DDL for index in the following cell:

In [ ]:
%%sql
select pg_get_indexdef('idx_cities_city_name_range':: regclass);


Alternatively, use the `\d+` command:

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # \d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ idx_cities_city_name_range"

`USING lsm` indicates that the index is of the type Log-Structured Merge tree (LSM) and employs range sharding. A LSM tree is not an tree structure, but rather, a complex algorithm that converts discrete random write requests into batch sequential write requests. To improve write performance for the LSM tree, RocksDB utilizes a Write-Ahead Log (WAL) and a memtable (a skiplist that lives in memory). The sequential writes in the memtable persists to disk as a Sorted Sequence Table (SSTable or SST) files.

Both hash sharding and range sharding use LSM. The difference of course is that the LSM tree sorts the hash coded values for hash sharding and the specified key values for range sharding.

> Important!
> 
> In Data Definition Language statements, YugabyteDB will interpret the `BTREE` keyword as `LSM`. YugabyteDB does not support `BTREE` indexes.

### View the Index details in the YB-Master web ui
You can view the details of the `idx_cities_city_name_range` index in the YB-Master web ui. Run the cell below and open the URL in your web browser.

In [ ]:
#%% python, but prepared statements as sql magic
WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
view_url = %sql select fn_get_table_id(:WORKSPACE_URL,8200,'idx_cities_city_name_range') as open_this_url_in_your_web_browser;

print (view_url)

#### Review the Column section
The column section shows details about each column in the index. Here is the section for  `idx_cities_city_name_range`:


| Column | ID	| Type |
|--------|------|------|
| city_name    | 0	| string NOT NULL NOT A PARTITION KEY | 
| ybidxbasectid	   | 1	| binary NOT NULL NOT A PARTITION KEY | 


<br/>

> Important!
>  
>  YugabyteDB creates an internal, hidden column, `ybidxbasectid`, for the indexed row. `ybidxbasectid` is similar to the internal, hidden colum, `ybctid`, for a row of a table. Both `ybctid` and  `ybidxbasectid` are virtual columns that represent the
>  DocDB-encoded key for the tuple. 
> 
> Using  `\d` or `\d+` will not show the `ybidxbasectid` column. It is also not possible to query the `ybidxbasectid` value.

#### Review the Tablet section
The Tablet section shows the details for the existing tablets. Here is the section for  `idx_cities_city_name_range`: 

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1<br>`1e2c3ef228534d3cbbf59c9fa6968d88	` |	`range: [<start>, <end>)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.1 |

YugabyteDB will automatically split this tablet based on the size of the table on disk. The following global flags determine this behavior:

```
--tablet_force_split_threshold_bytes=107374182400 --> 10240 MB
--tablet_split_high_phase_shard_count_per_node=24
--tablet_split_high_phase_size_threshold_bytes=10737418240 --> 10240 MB
--tablet_split_low_phase_shard_count_per_node=8
--tablet_split_low_phase_size_threshold_bytes=536870912 --> 512 MB
--tablet_split_size_threshold_bytes=0
```

The low phase indicates the threshold for the initial splits of the tablet. With more data volume, the threshold increases from 512 MB to over 10 GBs.

### q1a | Range index in equality predicate
To generate an Explain Plan, run the following cell:

In [ ]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 , *
 --, city_id
 --, city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
and city_name = 'Alameda'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
-- limit 100
;

#### q1a | Explain Plan (above ^^)

The Explain Plan shows that this query uses the index and reads 2 rows
- `Index Scan using idx_cities_city_name_range on public.tbl_cities (actual time=6.392..6.397 rows=2 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: ((tbl_cities.city_name)::text = 'Alameda'::text)`

To view the tablet metrics, run the following cell:

In [88]:
%%sql
execute  stmt_util_metrics_snap_table;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
4 rows affected.


[dbname | relname | tableid | tabletid | isLeader],rocksdb_number_db_seek,rocksdb_number_db_next,rows_inserted
db_ybu | idx_cities_city_name_range | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004686 | 6a230325f23b4199a60fb6dd9659008a | L,1,16,None
db_ybu | tbl_cities | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004039 | 3f57518dc4dd4b3f9171706d84522512 | L,18,126,None
db_ybu | tbl_cities | http://127.0.0.2:7000/table?id=00004000000030008000000000004039 | e92b77a0c8c84f95b3d68586df630ad0 | L,8,56,None
db_ybu | tbl_cities | http://127.0.0.3:7000/table?id=00004000000030008000000000004039 | 9bfa7ea6610c402ba08648f47652e93c | L,6,42,None


#### q1a | Metrics (above ^^)

In the initial query, the `Index Scan` accesses the index tablet:

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu idx_cities_city_name_range link_table_id tablet_id_unq_1 leader	 | 1| 	2 |

There are two results for the predicate expression, resulting in one seek of the index tablet offset, and then two reads.
Because the query returns all columns, the query also reads from one of the tablet leader for `tbl_cities`, a table with hash sharding.


| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_cities link_table_id tablet_id_unq_1	| 4 | 28 |


What the Metrics report reveals is that even though the query uses the range index, the query must still seek 4 offsets from one of the tablets for `tbl_cities`.



###  q1b | Covering index with an equality predicate
When the query results include all the columns in the index, the index "covers the query" and is known as a "covering index".

To view the Explain Plan, run the following cell: 

In [ ]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 --, *
-- , city_id
 , city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
and city_name = 'Alameda'
-- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
-- limit 100
;

#### q1b | Explain plan (above ^^)
The Explain Plan shows that this query uses the index only `Index Only Scan` and reads 2 rows.
- `Index Only Scan using idx_cities_city_name_range on public.tbl_cities (actual time=4.964..4.968 rows=2 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: ((tbl_cities.city_name)::text = 'Alameda'::text)`

To view the tablet metrics, run the following cell:

In [ ]:
%%sql
execute  stmt_util_metrics_snap_table;

#### q1b | Metrics (above ^^)

In the initial query, the `Index Scan` accesses the index tablet:

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu idx_cities_city_name_range link_table_id tablet_id_unq_1 leader	 | 1| 	2 |

There are two results for the predicate expression, resulting in one seek of the index tablet offset, and then two reads.
Because the query only returns the city name, the index "covers" the query and is a "coverign index". There is no need for the query to access the tablets for `tbl_cities`.


#### q1b | Experiment

Question: What happens when the query includes the PK column, `city_id`,  in the select command? 

Answer:
  - The Explain Plan becomes an `Index Scan`
  - The query eads from a tablet for `tbl_cities`
  - The reason? The tuple for the index, `ybidxbasectid`, encodes the PK pointer. YugabyteDB at this time is unable to use this PK encoding in results. This is a known issue.

Question: What happens the query includes a PK equality clause in the predicate?

Answer:
- The query will use the PK index, `tbl_cities_pkey`, and not the secondary index.



### q1c | Range index with a range predicate
Run the following cell to view the query results:

In [ ]:
%%sql
-- to see results
select '' _
   , *
-- , city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
 and city_name BETWEEN 'Alameda' AND  'Alamo'
-- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
-- and yb_hash_code(city_name) = yb_hash_code(city_name::text)
limit 30
;

To view the Explain Plan, run the following cell:

In [ ]:
%%sql

execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze, verbose) 
select ''_ 
    , *
 -- , city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
 and city_name BETWEEN 'Alameda' AND  'Alamo'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
-- and yb_hash_code(city_name) = yb_hash_code(city_name::text)
-- limit 100
;

#### q1c | Explain plan (above ^^)
The Explain Plan shows that this query uses the index only `Index Scan` and reads 16 rows.
- `Index Scan using idx_cities_city_name_range on public.tbl_cities (actual time=3.192..3.239 rows=16 loops=1`

The Index Condition reflects the query predicate range.

- `  Index Cond: (((tbl_cities.city_name)::text >= 'Alameda'::text) AND ((tbl_cities.city_name)::text <= 'Alamo'::text))`

To view the tablet metrics, run the following cell:

In [ ]:
%%sql

execute  stmt_util_metrics_snap_table;

#### q1c | Metrics (above ^^)

In the initial query, the `Index Scan` accesses the index tablet:

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu idx_cities_city_name_range link_table_id tablet_id_unq_1 leader	 | 1 | 	16 |

Because the query returns all the columns, the query accesses data on the the three tablet leaders for `tbl_cities`. Retrieving the column data for the query requires additional seeks and reads on each tablet.


### q1d | Covering index with a range predicate
To view the Explain Plan, run the following cell:

In [78]:
%%sql

execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 --, *
 -- , city_id
 , city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
and city_name BETWEEN 'Alameda' AND 'Alamo'
-- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
-- limit 100
;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
7 rows affected.


QUERY PLAN
Index Only Scan using idx_cities_city_name_range on public.tbl_cities (actual time=0.883..0.891 rows=16 loops=1)
"Output: ''::text, city_name"
Index Cond: ((tbl_cities.city_name >= 'Alameda'::text) AND (tbl_cities.city_name <= 'Alamo'::text))
Heap Fetches: 0
Planning Time: 0.092 ms
Execution Time: 0.922 ms
Peak Memory Usage: 8 kB


#### q1d | Explain Plan (above ^^)

The Explain Plan shows that this query only uses the range index and in doing so, retrieves 16 rows.
- `Index Only Scan using idx_cities_city_name_range on public.tbl_cities (actual time=0.883..0.891 rows=16 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: ((tbl_cities.city_name >= 'Alameda'::text) AND (tbl_cities.city_name <= 'Alamo'::text))`

To view the tablet metrics, run the following cell:

In [79]:
%%sql

execute  stmt_util_metrics_snap_table;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


[dbname | relname | tableid | tabletid | isLeader],rocksdb_number_db_seek,rocksdb_number_db_next,rows_inserted
db_ybu | idx_cities_city_name_range | http://127.0.0.3:7000/table?id=00004000000030008000000000004686 | 6a230325f23b4199a60fb6dd9659008a | L,1,16,None


#### q1d | Metrics (above ^^)

The range index covers the query and serves as a covering index. A covering index is indicative of a Index Only Scan query.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu idx_cities_city_name_range link_table_id tablet_id_unq_1 leader	 | 1 | 	16 |



---
## q2 | Create a secondary index using hash range sharding
In this series of queries, you will explore using a hash index. 

To begin, create an index with hash sharding. In the DLL for the index, specify the `hash` keyword as follows:

In [98]:
%%sql 
drop index if exists idx_cities_city_name_range;

drop index if exists idx_cities_city_name_hash;

select pg_sleep(1);

create index idx_cities_city_name_hash on tbl_cities (city_name hash);

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
Done.
Done.
1 rows affected.
Done.


[]

To view the DDL, run the following cell:

In [99]:
%%sql
select pg_get_indexdef('idx_cities_city_name_hash':: regclass);

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


pg_get_indexdef
CREATE INDEX idx_cities_city_name_hash ON public.tbl_cities USING lsm (city_name HASH)


### View the Index details in the YB-Master web ui
You can view the details of the `idx_cities_city_name_hash` index in the YB-Master web ui. Run cell below and open the URL in your web browser.

In [94]:
#%% python, but prepared statements as sql magic
WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
view_url = %sql select fn_get_table_id(:WORKSPACE_URL,8200,'idx_cities_city_name_hash') as open_this_url_in_your_web_browser;

print (view_url)

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
+--------------------------------------------------------------------------------------------------------------+
|                                      open_this_url_in_your_web_browser                                       |
+--------------------------------------------------------------------------------------------------------------+
| https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=000040000000300080000000000046eb |
+--------------------------------------------------------------------------------------------------------------+


#### Review the Column section
The column section shows details about each column in the index. Here is the section for  `idx_cities_city_name_hash`:


| Column | ID	| Type |
|--------|------|------|
| city_name    | 0	| string NOT NULL PARTITION KEY| 
| ybidxbasectid	   | 1	| binary NOT NULL NOT A PARTITION KEY | 


<br/>

> Important!
>  
>  YugabyteDB creates an internal, hidden column, `ybidxbasectid`, for the indexed row. `ybidxbasectid` is similar to the internal, hidden colum, `ybctid`, for a row of a table. Both `ybctid` and  `ybidxbasectid` are virtual columns that represent the
>  DocDB-encoded key for the tuple. 
> 
> Using  `\d` or `\d+` will not show the `ybidxbasectid` column. It is also not possible to query the `ybidxbasectid` value.

##### Partition Key
YugabyteDB uses the shard key (shown as `PARTITION KEY`) to distribute the data among the tablet leaders for the index. 

With consistent hash sharding, a partitioning algorithm distributes data evenly and randomly across shards. By computing a consistent hash on the partition key (or keys) of a given row, YugabyteDB knows where to insert the row among the tablet leaders.



#### Review the Tablet section
For the given index, the Tablet section shows the details for the existing tablets. Of particular interest are the number of tablet leaders and the partition strategy. Here is an example of the Tablet section for `idx_cities_city_name_hash`:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1 |	`hash_split: [0x5555, 0xAAAA)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |
| some_uuid_2	| `hash_split: [0xAAAA, 0xFFFF)`	| 0 |  Running |false |	Tablet reported with an active leader |<li>FOLLOWER: 127.0.0.1 <li>LEADER: 127.0.0.3 <li>FOLLOWER: 127.0.0.2 |
| some_uuid_3 <br>(tablet leader where the row lives) |	`hash_split: [0x0000, 0x5555)` |	0 |	Running | 	false	| Tablet reported with an active leader |	<li>LEADER: 127.0.0.1<li>FOLLOWER: 127.0.0.3<li>FOLLOWER: 127.0.0.2 |

### q2a | Hash index with an equality predicate
To view the Explain Plan for the query, run the following cell:

In [100]:
%%sql

execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select 0
 , *
 -- , city_id
 -- , city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
and city_name = 'Alameda'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
-- limit 100
;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
6 rows affected.


QUERY PLAN
Index Scan using idx_cities_city_name_hash on public.tbl_cities (actual time=1.766..1.772 rows=2 loops=1)
"Output: 0, city_id, city_name, city_name_alt, state_id, state_code, country_id, country_code, latitude, longitude, created_at, updated_at, flag, wiki_data_id"
Index Cond: ((tbl_cities.city_name)::text = 'Alameda'::text)
Planning Time: 2.497 ms
Execution Time: 1.821 ms
Peak Memory Usage: 8 kB


#### q2a | Explain Plan (above ^^)

The Explain Plan shows that this query uses the hash index and reads 2 rows.
- `Index Scan using idx_cities_city_name_hash on public.tbl_cities (actual time=3.539..3.552 rows=2 loops=1`

The Index Condition reflects the query predicate.

- `Index Cond: ((tbl_cities.city_name)::text = 'Alameda'::text)`

To view the tablet metrics, run the following cell:

In [101]:
%%sql

execute  stmt_util_metrics_snap_table;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
2 rows affected.


[dbname | relname | tableid | tabletid | isLeader],rocksdb_number_db_seek,rocksdb_number_db_next,rows_inserted
db_ybu | idx_cities_city_name_hash | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004752 | 94bde5cf6e2940c0839d4145fc33e9b6 | L,1,2,None
db_ybu | tbl_cities | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004039 | 3f57518dc4dd4b3f9171706d84522512 | L,4,28,None


#### q2a | Metrics (above ^^)

The query uses the hash index and accesses one of the tablet leaders for `idx_cities_city_name_hash`.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu idx_cities_city_name_hash link_table_id tablet_id_unq_1 leader	 | 1 | 	2 |

Because the query requires more than the columns in the index, the query accesses one of the tablet leaders for `tbl_cities`.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_cities link_table_id tablet_id_unq_1 leader	 | 4 | 	28 |

There's a 33% chance that the tablet leaders for the index and the table are on different hosts. If you are running this notebook locally or in Gitpod, the host is the same machine with the YB-TServer processes running on different ports using host aliases for localhost (127.0.0.1) such as 127.0.0.2 and 127.0.0.3.


### q2b | Covering index with a hash index and equality predicate
Run the following cell to generate an Explain Plan:

In [102]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 -- , *
 --, city_id
, city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
and city_name = 'Alameda'
 -- and city_name BETWEEN 'Alameda' AND 'Alamo'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
-- limit 100
;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
7 rows affected.


QUERY PLAN
Index Only Scan using idx_cities_city_name_hash on public.tbl_cities (actual time=0.576..0.578 rows=2 loops=1)
"Output: ''::text, city_name"
Index Cond: (tbl_cities.city_name = 'Alameda'::text)
Heap Fetches: 0
Planning Time: 0.559 ms
Execution Time: 0.607 ms
Peak Memory Usage: 8 kB


#### q2a | Explain Plan (above ^^)

The Explain Plan shows that the index is a covering index and returns 2 rows.
- `Index Only Scan using idx_cities_city_name_hash on public.tbl_cities (actual time=0.805..0.808 rows=2 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: (tbl_cities.city_name = 'Alameda'::text)`

To view the tablet metrics, run the following cell:

In [103]:
%%sql
execute  stmt_util_metrics_snap_table;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


[dbname | relname | tableid | tabletid | isLeader],rocksdb_number_db_seek,rocksdb_number_db_next,rows_inserted
db_ybu | idx_cities_city_name_hash | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004752 | 94bde5cf6e2940c0839d4145fc33e9b6 | L,1,2,None


#### q2b | Metrics (above ^^)

As an Index Only Scan query, the query uses the hash index and accesses one of the tablet leaders for `idx_cities_city_name_hash` and reads 2 rows.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu idx_cities_city_name_hash link_table_id tablet_id_unq_1 leader	 | 1 | 	2 |


### q2c | Hash index with a range predicate
To view the Explain Plan, run the following:

In [104]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
    , *
-- , city_id
 --, city_name
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
and city_name BETWEEN 'Alameda' AND 'Alamo'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
-- limit 100
;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
7 rows affected.


QUERY PLAN
Seq Scan on public.tbl_cities (actual time=366.921..2929.900 rows=16 loops=1)
"Output: ''::text, city_id, city_name, city_name_alt, state_id, state_code, country_id, country_code, latitude, longitude, created_at, updated_at, flag, wiki_data_id"
Filter: (((tbl_cities.city_name)::text >= 'Alameda'::text) AND ((tbl_cities.city_name)::text <= 'Alamo'::text))
Rows Removed by Filter: 148250
Planning Time: 0.092 ms
Execution Time: 2929.967 ms
Peak Memory Usage: 8 kB


#### q2c | Explain plan (above ^^)
As expected, the optimizer generates an Explain Plan that does not use the hash index. Instead, the Explain Plan shows a full table scan of the table itself.
- `Seq Scan on public.tbl_cities (actual time=366.921..2929.900 rows=16 loops=1`

The YB-TServer that serves the client connection removes the rows:
  - `Filter: (((tbl_cities.city_name)::text >= 'Alameda'::text) AND ((tbl_cities.city_name)::text <= 'Alamo'::text))`
  - `Rows Removed by Filter: 148250`

To view the Metrics report for the query, run the following cell:


In [105]:
%%sql

execute  stmt_util_metrics_snap_table;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
3 rows affected.


[dbname | relname | tableid | tabletid | isLeader],rocksdb_number_db_seek,rocksdb_number_db_next,rows_inserted
db_ybu | tbl_cities | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004039 | 3f57518dc4dd4b3f9171706d84522512 | L,34461,673947,None
db_ybu | tbl_cities | http://127.0.0.2:7000/table?id=00004000000030008000000000004039 | e92b77a0c8c84f95b3d68586df630ad0 | L,34597,678580,None
db_ybu | tbl_cities | http://127.0.0.3:7000/table?id=00004000000030008000000000004039 | 9bfa7ea6610c402ba08648f47652e93c | L,34677,680376,None


#### q2b | Metrics (above ^^)
The `Seq Scan` requires that the query access all three tablets for `tbl_cities`, requiring 30K+ seeks and 60OK reads per tablet. 

### q5 | Cleanup
Drop index

In [ ]:
%%sql
drop index if exists idx_cities_city_name_hash;
drop index if exists idx_cities_city_name_range;

---
## Review

PK in predicate will take precedence and result in index NOT being used...
- Can't drop or alter PK, need to rename, create new table (or similar), and select into to make change.
- PK can be composite (hash, and range order), and this behavior can be different if that is the case [not covered here]

Secondary index as Range 
- Equality is performant, but when range index tablets split, may not be as great as hash index for equality
- Ideal for range and comparison predicates
- For both Equality and Comparison predicate...
  - When the index column is the sole column in the select command and the sole column in the predicate (no PK), the range index is the covering index, and results in Index Only Scan
 
Secondary index as Hash
- Equality is ideal
  -  When the index column is the sole column in the select command and the sole column in the predicate (no PK), the range index is the covering index, and results in Index Only Scan
- A range or comparison predicate ignores the hash index and results in a costly Seq Scan

YugabyteDB vs PostgreSQL
- YugabyteDB only updates the indexes of columns where there are key-values and subkey values changes.
- A Index Only Scan reads from the index tablet.

---
## q3 | Expression index
uses expression for index

In [ ]:
%%sql
drop index if exists idx_cities_city_name_exp;

create index idx_cities_city_name_exp on tbl_cities (UPPER (COALESCE(city_name, city_name_alt) ) asc);

### q3a | Expression query

In [ ]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 -- , *
, city_id
, city_name
, city_name_alt
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coue = 'CA'
 and UPPER (COALESCE(city_name, city_name_alt) ) like 'A%'
 -- and city_name = 'Alameda'
 -- and city_name BETWEEN 'Alameda' AND 'Alamo'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
limit 100
;

#### q3a | Explain plan (above ^^)
- Index is Range
- Index Scan
- Expression in Index Cond --> This shows the boundary >='A' <'B'
- Filter not doing anything



In [ ]:
%%sql

execute  stmt_util_metrics_snap_table;

####  | Metrics (above ^^)
- Only index tablet, which is range
- Accesses the other table tablets

---
## q4 | Create secondary index using expression and include
uses Include

In [ ]:
%%sql
drop index if exists idx_cities_city_name_exp_covering;

create index idx_cities_city_name_covering on tbl_cities (UPPER (COALESCE(city_name, city_name_alt) ) asc) include (city_id, city_name, city_name_alt);

### q4a | Query using expression and include columns

In [ ]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 -- , *
, city_id
, city_name
, city_name_alt
-- , state_id
from tbl_cities 
where 1=1 
 -- and country_id = 233
 -- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
 and UPPER (COALESCE(city_name, city_name_alt) ) like 'A%'
 -- and city_name = 'Alameda'
 -- and city_name BETWEEN 'Alameda' AND 'Alamo'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
limit 100
;

#### q4a | Explain plan (above ^^)
- Index is Range
- Index Only Scan
- Expression in Index Cond --> This shows the boundary >='A' <'B'
- Filter not doing anything



In [ ]:
%%sql

execute  stmt_util_metrics_snap_table;

#### q4a | Metrics (above ^^)
- Only index tablet means 1 rpc
- Trade off is data storage
- Non-DQL access patterns, such as DML [insert, update, delete]... Impact comes if source table is highly touched


---
## q5 | Partial index
uses WHERE
- TODO: for comparison

In [ ]:
%%sql
drop index if exists idx_cities_city_name_covering_US;

create index idx_cities_city_name_covering_US
on tbl_cities (UPPER (COALESCE(city_name, city_name_alt) ) asc) 
include (city_id, city_name, city_name_alt) 
where country_code='US';

### q5a | Create secondary index using expression and include and where

In [ ]:
%%sql
execute  stmt_util_metrics_snap_reset;

explain (costs off, analyze,verbose) 
select '' _
 -- , *
, city_id
, city_name
, city_name_alt
from tbl_cities 
where 1=1 
 -- and country_id = 233
 and country_code = 'US'
-- and country_code = 'US'
 -- and state_id = 1416
 -- and state_coude = 'CA'
 and UPPER (COALESCE(city_name, city_name_alt) ) like 'A%'
 -- and city_name = 'Alameda'
 -- and city_name BETWEEN 'Alameda' AND 'Alamo'
 -- and city_id = 111088
 -- and yb_hash_code(city_id) = yb_hash_code(111088)
 -- and yb_hash_code(city_name) = yb_hash_code('Alameda'::text)
limit 100
;

#### q5 | Explain plan (above ^^)
- Index is Range
- Index Only Scan
- Expression in Index Cond
  - boundary >='A' <'B'
  - where is not shown
- Filter not doing anything, as index cond handles


In [ ]:
%%sql

execute  stmt_util_metrics_snap_table;

#### q8 | Metrics (above ^^)
- Only index tablet means 1 rpc
- Trade off is data storage
- Non-DQL access patterns, such as DML [insert, update, delete]... Impact comes if source table is highly touched


---
https://sourcegraph.com/github.com/yugabyte/yugabyte-db/-/blob/src/yb/common/pg_system_attr.h?L30&subtree=true

```
namespace yb {

// In addition to regular columns, YB support for postgres also have virtual columns.
// Virtual columns are just expression that is evaluated by DocDB in "doc_expr.cc".
enum class PgSystemAttrNum : int {
  // Postgres system columns.
  kSelfItemPointer      = -1, // ctid.
  kObjectId             = -2, // oid.
  kMinTransactionId     = -3, // xmin
  kMinCommandId         = -4, // cmin
  kMaxTransactionId     = -5, // xmax
  kMaxCommandId         = -6, // cmax
  kTableOid             = -7, // tableoid

  // YugaByte system columns.
  kYBTupleId            = -8, // ybctid: virtual column representing DocDB-encoded key.
                              // YB analogue of Postgres's SelfItemPointer/ctid column.

  // The following attribute numbers are stored persistently in the table schema. For this reason,
  // they are chosen to avoid potential conflict with Postgres' own sys attributes now and future.
  kYBRowId              = -100, // ybrowid: auto-generated key-column for tables without pkey.
  kYBIdxBaseTupleId     = -101, // ybidxbasectid: for indexes ybctid of the indexed table row.
  kYBUniqueIdxKeySuffix = -102, // ybuniqueidxkeysuffix: extra key column for unique indexes, used
                                // to ensure SQL semantics for null (null != null) in DocDB
                                // (where null == null). For each index row will be set to:
                                //  - the base table ctid when one or more indexed cols are null
                                //  - to null otherwise (all indexed cols are non-null).
};

} // namespace yb
```

https://sourcegraph.com/search?q=context:global+repo:%5Egithub%5C.com/yugabyte/yugabyte-db%24+ybctid+file:%5Esrc/yb/&patternType=literal